In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile1 = pd.read_pickle("FFF_Train_Decile1.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile1.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile1, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 1s 442us/step - loss: 0.5619 - acc: 0.7146 - val_loss: 0.5383 - val_acc: 0.7395
Epoch 38/100
3055/3055 [==============================] - 1s 471us/step - loss: 0.5650 - acc: 0.7218 - val_loss: 0.5379 - val_acc: 0.7356
Epoch 39/100
3055/3055 [==============================] - 1s 424us/step - loss: 0.5668 - acc: 0.7231 - val_loss: 0.5383 - val_acc: 0.7369
Epoch 40/100
3055/3055 [==============================] - 1s 422us/step - loss: 0.5648 - acc: 0.7175 - val_loss: 0.5377 - val_acc: 0.7369
Epoch 41/100
3055/3055 [==============================] - 1s 428us/step - loss: 0.5614 - acc: 0.7149 - val_loss: 0.5365 - val_acc: 0.7382
Epoch 42/100
3055/3055 [==============================] - 1s 428us/step - loss: 0.5643 - acc: 0.7162 - val_loss: 0.5361 - val_acc: 0.7369
Epoch 43/100
3055/3055 [==============================] - 1s 440us/step - loss: 0.5594 - acc: 0.7201 - val_loss: 0.5364 - val_acc: 0.7317
Epoch 44/100
3055/3055 [=======================

Epoch 96/100
3055/3055 [==============================] - 1s 428us/step - loss: 0.5544 - acc: 0.7191 - val_loss: 0.5311 - val_acc: 0.7356
Epoch 97/100
3055/3055 [==============================] - 1s 430us/step - loss: 0.5572 - acc: 0.7185 - val_loss: 0.5311 - val_acc: 0.7356
Epoch 98/100
3055/3055 [==============================] - 1s 428us/step - loss: 0.5534 - acc: 0.7241 - val_loss: 0.5310 - val_acc: 0.7356
Epoch 99/100
3055/3055 [==============================] - 1s 425us/step - loss: 0.5565 - acc: 0.7211 - val_loss: 0.5309 - val_acc: 0.7356
Epoch 100/100
3055/3055 [==============================] - 1s 429us/step - loss: 0.5597 - acc: 0.7211 - val_loss: 0.5310 - val_acc: 0.7356
Test accuracy: 0.7356020936167053
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 2s 732us/step - loss: 0.6267 - acc: 0.6514 - val_loss: 0.5719 - val_acc: 0.6846
Epoch 2/150
3055/3055 [==============================] - 1s 434us/step - loss: 0.5853 - acc

3055/3055 [==============================] - 1s 433us/step - loss: 0.5525 - acc: 0.7247 - val_loss: 0.5256 - val_acc: 0.7369
Epoch 54/150
3055/3055 [==============================] - 1s 455us/step - loss: 0.5503 - acc: 0.7227 - val_loss: 0.5256 - val_acc: 0.7369
Epoch 55/150
3055/3055 [==============================] - 1s 445us/step - loss: 0.5488 - acc: 0.7247 - val_loss: 0.5256 - val_acc: 0.7369
Epoch 56/150
3055/3055 [==============================] - 1s 435us/step - loss: 0.5476 - acc: 0.7211 - val_loss: 0.5257 - val_acc: 0.7369
Epoch 57/150
3055/3055 [==============================] - 1s 425us/step - loss: 0.5479 - acc: 0.7237 - val_loss: 0.5256 - val_acc: 0.7369
Epoch 58/150
3055/3055 [==============================] - 1s 415us/step - loss: 0.5487 - acc: 0.7195 - val_loss: 0.5255 - val_acc: 0.7369
Epoch 59/150
3055/3055 [==============================] - 1s 435us/step - loss: 0.5473 - acc: 0.7277 - val_loss: 0.5254 - val_acc: 0.7369
Epoch 60/150
3055/3055 [=======================

3055/3055 [==============================] - 2s 506us/step - loss: 0.5483 - acc: 0.7227 - val_loss: 0.5254 - val_acc: 0.7369
Epoch 113/150
3055/3055 [==============================] - 2s 495us/step - loss: 0.5523 - acc: 0.7273 - val_loss: 0.5254 - val_acc: 0.7369
Epoch 114/150
3055/3055 [==============================] - 1s 488us/step - loss: 0.5493 - acc: 0.7205 - val_loss: 0.5253 - val_acc: 0.7369
Epoch 115/150
3055/3055 [==============================] - 2s 503us/step - loss: 0.5487 - acc: 0.7205 - val_loss: 0.5253 - val_acc: 0.7369
Epoch 116/150
3055/3055 [==============================] - 2s 510us/step - loss: 0.5483 - acc: 0.7208 - val_loss: 0.5253 - val_acc: 0.7369
Epoch 117/150
3055/3055 [==============================] - 2s 502us/step - loss: 0.5482 - acc: 0.7214 - val_loss: 0.5253 - val_acc: 0.7369
Epoch 118/150
3055/3055 [==============================] - 1s 482us/step - loss: 0.5483 - acc: 0.7270 - val_loss: 0.5253 - val_acc: 0.7369
Epoch 119/150
3055/3055 [================

...

Epoch 6/200
3055/3055 [==============================] - 1s 445us/step - loss: 0.5666 - acc: 0.7149 - val_loss: 0.5374 - val_acc: 0.7317
Epoch 7/200
3055/3055 [==============================] - 1s 437us/step - loss: 0.5640 - acc: 0.7165 - val_loss: 0.5351 - val_acc: 0.7356
Epoch 8/200
3055/3055 [==============================] - 1s 429us/step - loss: 0.5631 - acc: 0.7169 - val_loss: 0.5365 - val_acc: 0.7291
Epoch 9/200
3055/3055 [==============================] - 1s 432us/step - loss: 0.5624 - acc: 0.7119 - val_loss: 0.5331 - val_acc: 0.7382
Epoch 10/200
3055/3055 [==============================] - 1s 441us/step - loss: 0.5566 - acc: 0.7247 - val_loss: 0.5376 - val_acc: 0.7291
Epoch 11/200
3055/3055 [==============================] - 1s 447us/step - loss: 0.5581 - acc: 0.7247 - val_loss: 0.5428 - val_acc: 0.7225
Epoch 12/200
3055/3055 [==============================] - 2s 508us/step - loss: 0.5546 - acc: 0.7205 - val_loss: 0.5330 - val_acc: 0.7343
Epoch 13/200
3055/3055 [==============

Epoch 64/200
3055/3055 [==============================] - 1s 439us/step - loss: 0.5534 - acc: 0.7221 - val_loss: 0.5291 - val_acc: 0.7369
Epoch 65/200
3055/3055 [==============================] - 1s 443us/step - loss: 0.5514 - acc: 0.7227 - val_loss: 0.5290 - val_acc: 0.7356
Epoch 66/200
3055/3055 [==============================] - 1s 457us/step - loss: 0.5529 - acc: 0.7165 - val_loss: 0.5289 - val_acc: 0.7343
Epoch 67/200
3055/3055 [==============================] - 1s 437us/step - loss: 0.5527 - acc: 0.7195 - val_loss: 0.5289 - val_acc: 0.7356
Epoch 68/200
3055/3055 [==============================] - 1s 431us/step - loss: 0.5499 - acc: 0.7234 - val_loss: 0.5289 - val_acc: 0.7343
Epoch 69/200
3055/3055 [==============================] - 1s 431us/step - loss: 0.5504 - acc: 0.7227 - val_loss: 0.5289 - val_acc: 0.7356
Epoch 70/200
3055/3055 [==============================] - 1s 444us/step - loss: 0.5534 - acc: 0.7185 - val_loss: 0.5288 - val_acc: 0.7343
Epoch 71/200
3055/3055 [==========

3055/3055 [==============================] - 1s 433us/step - loss: 0.5511 - acc: 0.7244 - val_loss: 0.5286 - val_acc: 0.7369
Epoch 124/200
3055/3055 [==============================] - 1s 440us/step - loss: 0.5524 - acc: 0.7270 - val_loss: 0.5285 - val_acc: 0.7369
Epoch 125/200
3055/3055 [==============================] - 1s 428us/step - loss: 0.5509 - acc: 0.7205 - val_loss: 0.5287 - val_acc: 0.7382
Epoch 126/200
3055/3055 [==============================] - 1s 428us/step - loss: 0.5525 - acc: 0.7221 - val_loss: 0.5286 - val_acc: 0.7369
Epoch 127/200
3055/3055 [==============================] - 1s 431us/step - loss: 0.5449 - acc: 0.7224 - val_loss: 0.5286 - val_acc: 0.7369
Epoch 128/200
3055/3055 [==============================] - 1s 459us/step - loss: 0.5535 - acc: 0.7224 - val_loss: 0.5286 - val_acc: 0.7369
Epoch 129/200
3055/3055 [==============================] - 1s 434us/step - loss: 0.5505 - acc: 0.7175 - val_loss: 0.5286 - val_acc: 0.7369
Epoch 130/200
3055/3055 [================

3055/3055 [==============================] - 1s 431us/step - loss: 0.5530 - acc: 0.7185 - val_loss: 0.5284 - val_acc: 0.7382
Epoch 183/200
3055/3055 [==============================] - 1s 426us/step - loss: 0.5520 - acc: 0.7234 - val_loss: 0.5284 - val_acc: 0.7369
Epoch 184/200
3055/3055 [==============================] - 1s 436us/step - loss: 0.5546 - acc: 0.7218 - val_loss: 0.5283 - val_acc: 0.7382
Epoch 185/200
3055/3055 [==============================] - 1s 433us/step - loss: 0.5479 - acc: 0.7234 - val_loss: 0.5284 - val_acc: 0.7382
Epoch 186/200
3055/3055 [==============================] - 1s 430us/step - loss: 0.5528 - acc: 0.7241 - val_loss: 0.5283 - val_acc: 0.7382
Epoch 187/200
3055/3055 [==============================] - 1s 438us/step - loss: 0.5511 - acc: 0.7218 - val_loss: 0.5283 - val_acc: 0.7382
Epoch 188/200
3055/3055 [==============================] - 1s 461us/step - loss: 0.5481 - acc: 0.7250 - val_loss: 0.5283 - val_acc: 0.7382
Epoch 189/200
3055/3055 [================

3055/3055 [==============================] - 1s 407us/step - loss: 0.5536 - acc: 0.7178 - val_loss: 0.5270 - val_acc: 0.7395
Epoch 41/50
3055/3055 [==============================] - 1s 406us/step - loss: 0.5536 - acc: 0.7185 - val_loss: 0.5270 - val_acc: 0.7395
Epoch 42/50
3055/3055 [==============================] - 1s 400us/step - loss: 0.5517 - acc: 0.7188 - val_loss: 0.5270 - val_acc: 0.7395
Epoch 43/50
3055/3055 [==============================] - 1s 396us/step - loss: 0.5548 - acc: 0.7234 - val_loss: 0.5269 - val_acc: 0.7408
Epoch 44/50
3055/3055 [==============================] - 1s 400us/step - loss: 0.5494 - acc: 0.7208 - val_loss: 0.5269 - val_acc: 0.7395
Epoch 45/50
3055/3055 [==============================] - 1s 434us/step - loss: 0.5521 - acc: 0.7224 - val_loss: 0.5269 - val_acc: 0.7395
Epoch 46/50
3055/3055 [==============================] - 1s 433us/step - loss: 0.5489 - acc: 0.7211 - val_loss: 0.5270 - val_acc: 0.7395
Epoch 47/50
3055/3055 [==============================

In [3]:
0.93484

0.93484

In [ ]:
#0.749